In [1]:
import numpy as np
from codebase.file_utils import save_obj, load_obj, make_folder, path_backslash
from pdb import set_trace
from scipy.stats import multivariate_normal, norm
from scipy.optimize import minimize
from codebase.plot import *
alt.data_transformers.disable_max_rows()


DataTransformerRegistry.enable('default')

In [2]:
y_t = np.array([0,0,0,0,1,0])
theta_t=dict()
theta_t['alpha'] = np.array([0.3, 0.1, 0.4, 0.6, -0.8, 0.1])
theta_t['beta'] = np.ones((6,1))*0.3
z_t = np.array([[1]])


In [3]:
def get_pi_z(z, theta):
    exp_eta = np.exp(theta['alpha'] +  z @ theta['beta'].T)
    return exp_eta/(1+exp_eta)

pi_z = get_pi_z(z_t, theta_t)

In [4]:
def get_log_likelihood(z,y,theta):
    pi_z = get_pi_z(z, theta)
    s1 = np.sum((y*np.log(pi_z))+((1.-y)*(1.-np.log(pi_z))))
    s2 = -.5 * np.sum(z**2)
    return s1+s2

def get_neg_log_likelihood(z,y,theta):
    return - get_log_likelihood(z,y,theta)

def get_neg_posterior(z,y,theta):
    return - (get_log_likelihood(z,y,theta)+ norm.logpdf(z))


lglk = get_neg_log_likelihood(z_t, y_t, theta_t)
lglk

-5.733781394723231

In [5]:
def get_posterior_pdf(z, y, theta):
    return np.exp(get_log_likelihood(z ,y,theta) + norm.logpdf(z))
get_posterior_pdf(z_t, y_t, theta_t)

array([[74.80186827]])

In [37]:
def get_grad_pi_z(z, theta):
    exp_eta = np.exp(theta['alpha'] +  z @ theta['beta'].T)
    return (exp_eta *  theta['beta'].T)/(1+exp_eta)**2

def get_grad2_pi_z(z, theta):
    exp_eta = np.exp(theta['alpha'] +  z @ theta['beta'].T)
    return (-exp_eta *  (theta['beta'].T**2))/(1+exp_eta)**2

get_grad2_pi_z(z_t, theta_t)

array([[-0.02059058, -0.02162347, -0.01995416, -0.01849503, -0.02115033,
        -0.02162347]])

In [38]:
def get_grad2_log_likelihood(z,y,theta):
    pi_z = get_pi_z(z, theta)
    grad_pi_z = get_grad2_pi_z(z, theta)
    grad2_pi_z = get_grad2_pi_z(z, theta)
    
    r1 = (y/pi_z) - ((1. - y)/(1-pi_z))
    r2 = (y/pi_z**2) + ((1. - y)/(1-pi_z)**2)
    
    s1 = (grad2_pi_z*r1).sum() 
    s2 = (grad_pi_z**2*(r2)).sum() 
    return -1 + s1 - s2

get_grad2_log_likelihood(z_t, y_t, theta_t)

-0.7860582941183378

In [39]:
delta = 1e-5
z1 = z_t
z2 = z1 + delta
num_grad = (get_pi_z(z2, theta_t) - get_pi_z(z1, theta_t))/delta


In [40]:
if not np.allclose(num_grad, get_grad_pi_z(z1, theta_t)):
    print("Numerical Diff does not give the same answer as analytical")

In [41]:
def get_fisher_information(z, y, theta):
    pi_z = get_pi_z(z, theta)
    grad_pi_z = get_grad_pi_z(z, theta)
    r1 =grad_pi_z**2
    r2 =pi_z*(1.-pi_z)
    return 1. + np.sum(r1/r2)

get_fisher_information(z_t, y_t, theta_t)

1.1234370362299013

In [47]:
def get_hessian(z, y, theta):
    return -get_grad2_log_likelihood(z, y, theta)

get_hessian(z_t, y_t, theta_t)

0.7860582941183378

In [48]:
# try computing observed_information matrix = the hessian evaluated at the mode 

In [49]:
get_hessian(z_t, y_t, theta_t)

0.7860582941183378

In [50]:
get_fisher_information(z_t, y_t, theta_t)

1.1234370362299013

##

In [51]:
x0 = z_t.copy()*0.1
res = minimize(get_neg_log_likelihood, np.array([[1]]), args=(y_t, theta_t), method='BFGS')

In [52]:
def get_laplace_approx(y, theta, fisher=True):
    res = minimize(get_neg_posterior, np.array([[1]]), args=(y, theta), method='BFGS')
    if fisher:
        cov_matrix = get_fisher_information(res.x, y, theta)
    else:
        cov_matrix = get_hessian(res.x, y, theta)
    return multivariate_normal(mean = res.x, cov = cov_matrix**(-1))
lapldist =  get_laplace_approx(y_t, theta_t, False)
laplace_samples = lapldist.rvs(size = 10000)

In [53]:
df = pd.DataFrame(laplace_samples, columns=['value'])

In [54]:
laplace_pdf = alt.Chart(df).transform_fold(
        ['value']
        ).transform_density(
            density='value',
        ).mark_area(opacity=0.5).encode(
            alt.X('value:Q', title=None),
            alt.Y('density:Q')
)
laplace_pdf

alt.Chart(...)

In [55]:
prior_samples = norm.rvs(size = 10000)
df = pd.DataFrame(prior_samples, columns=['value'])
prior_pdf = alt.Chart(df).transform_fold(
        ['value']
        ).transform_density(
            density='value',
        ).mark_area(opacity=0.5, color='red').encode(
            alt.X('value:Q', title=None),
            alt.Y('density:Q')
)
prior_pdf

alt.Chart(...)

## tests

In [56]:
## numerical defrivative at argmax point should be zero
delta = 1e-3
z1 = res.x
z2 = z1 + delta
num_grad = (get_log_likelihood(z2, y_t, theta_t) - get_log_likelihood(z1, y_t,  theta_t))/delta
assert np.allclose(num_grad, 0., atol=1e-3)


In [57]:

lglks = []
for z_i in np.linspace(-4,4,40):
    l_i = get_posterior_pdf(z_i.reshape((1,1)), y_t, theta_t)
    lglks.append(l_i)

In [58]:
constant = np.squeeze(lglks).sum()*8/40.

In [59]:
df_lglks = pd.DataFrame(np.squeeze(lglks)/constant, columns=['post'])
df_lglks['z'] = np.linspace(-4,4,40)
alt.Chart(df_lglks).mark_line().encode(
            alt.X('z:Q', title=None),
            alt.Y('post:Q')
)+laplace_pdf+prior_pdf


alt.LayerChart(...)

In [22]:
def sigmoid(x, a, b):
    exp_x = np.exp(a +  x *b)
    return exp_x/(1.+exp_x)

def grad_sigmoid(x, a, b):
    exp_x = np.exp(a +  x *b)
    return b*exp_x/(1. + exp_x)**2

sigmoid(1. , 2. ,3)
grad_sigmoid(1. , 2. ,3)

0.019944170012370463

In [23]:
## numerical defrivative at argmax point should be zero
delta = 1e-3
z1 = 1.
z2 = z1 + delta
num_grad = (sigmoid(z2, 2, 3) - sigmoid(z1, 2, 3))/delta
assert np.allclose(num_grad, grad_sigmoid(z1 , 2. ,3), atol=1e-3)
# num_grad